In [ ]:
import time
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from ultralytics import YOLO

# =======================================
# CONFIGURAÇÕES GERAIS
# =======================================
DATASET_PATH = "../../LabelStudio/dataset/data.yaml"
IMAGE_SIZE = 640
EPOCHS = 5
BATCH_SIZE = 8
DEVICE = 0  # GPU = 0, CPU = "cpu"
RESULTS_DIR = Path("benchmark_results")
RESULTS_DIR.mkdir(exist_ok=True)

# Modelos YOLOv5 para testar
MODELS = {
    "yolov5n": "yolov5n.pt",
    "yolov5s": "yolov5s.pt",
    "yolov5m": "yolov5m.pt",
    "yolov5l": "yolov5l.pt",
    "yolov5x": "yolov5x.pt"
}

# Imagem de teste para medir inferência
TEST_IMAGE = "C:\\Users\\Usuario\\Desktop\\Faculdade\\LabelStudio\\dataset\\valid\\images\\frame_0005_jpg.rf.0cd077f418ce3010a8099a49a468ab2d.jpg"

# =======================================
# FUNÇÃO: Treinar modelo e medir tempo
# =======================================
def benchmark_train(model_name, model_path):
    model = YOLO(model_path)

    start = time.perf_counter_ns()
    results = model.train(
        data=DATASET_PATH,
        epochs=EPOCHS,
        imgsz=IMAGE_SIZE,
        batch=BATCH_SIZE,
        workers=1,
        device=DEVICE,
        patience=5,
        save=False,  # não salvar checkpoint
        pretrained=True
    )
    total_time = time.perf_counter_ns() - start
    time_per_epoch = total_time / EPOCHS
    return model, results, time_per_epoch

# =======================================
# FUNÇÃO: Avaliar métricas do modelo
# =======================================
def benchmark_eval(model):
    metrics = model.val()
    return {
        "precision": metrics.results_dict["metrics/precision(B)"],
        "recall": metrics.results_dict["metrics/recall(B)"],
        "mAP50": metrics.results_dict["metrics/mAP50(B)"],
        "mAP50_95": metrics.results_dict["metrics/mAP50-95(B)"]
    }

# =======================================
# FUNÇÃO: Medir tempo de inferência
# =======================================
def benchmark_infer(model):
    start = time.perf_counter_ns()
    _ = model(TEST_IMAGE)
    inference_time = time.perf_counter_ns() - start
    return inference_time

In [ ]:
# =======================================
# LOOP PRINCIPAL
# =======================================
results_list = []

for name, weights in MODELS.items():
    print(f"\n=== Benchmarking {name} ===")
    model, train_results, time_per_epoch = benchmark_train(name, weights)
    metrics = benchmark_eval(model)
    inference_time = benchmark_infer(model)

    results_list.append({
        "model": name,
        "time_per_epoch_ns": time_per_epoch,
        "precision": metrics["precision"],
        "recall": metrics["recall"],
        "mAP50": metrics["mAP50"],
        "mAP50_95": metrics["mAP50_95"],
        "inference_time_ns": inference_time
    })

In [ ]:
# =======================================
# AGRUPAR RESULTADOS EM DATAFRAME
# =======================================
df = pd.DataFrame(results_list)
print("\nResultados brutos:")
print(df)

# Salvar em parquet
df.to_parquet(RESULTS_DIR / "yolov5_benchmark.parquet", index=False)

In [ ]:
# =======================================
# PLOTS COM SEABORN
# =======================================
sns.set(style="whitegrid")

# Tempo por época
plt.figure(figsize=(8, 5))
sns.barplot(data=df, x="model", y="time_per_epoch")
plt.title("Tempo médio por época (nanosegundos)")
plt.show()

# Métricas de avaliação
melted = df.melt(id_vars=["model"], value_vars=["precision", "recall", "mAP50", "mAP50_95"])
plt.figure(figsize=(10, 6))
sns.barplot(data=melted, x="model", y="value", hue="variable")
plt.title("Métricas de avaliação por modelo")
plt.show()

# Tempo de inferência
plt.figure(figsize=(8, 5))
sns.barplot(data=df, x="model", y="inference_time")
plt.title("Tempo de inferência (nanosegundos por imagem)")
plt.show()